In [ ]:
from astropy import time
import numpy as np
from IPython import display
import pandas as pd
import panel as pn
pn.extension("tabulator")
import hvplot.pandas
import holoviews as hv

from lwautils import lwa_arx
from lwautils import TimeoutException
from lwa_antpos import reading, mapping
from mnc import mcs, common, control
import dsautils.cnf as cnf
from dsautils import dsa_store
from lwa_f import snap2_fengine, snap2_feng_etcd_client
from lwa352_pipeline_control import Lwa352PipelineControl, Lwa352CorrelatorControl

In [ ]:
ls = dsa_store.DsaStore()

## Set up

In [ ]:
con = control.Controller("/home/ubuntu/proj/lwa-shell/mnc_python/config/lwa_config_calim.yaml")

adrs = con.conf['arx']['adrs']
snap2names = con.conf['fengines']['snap2s_inuse']
xhosts = con.conf['xengines']['xhosts']
recorders = con.conf['dr']['recorders']

In [ ]:
dd_statuses = []

## cnf

In [ ]:
df_cnf_read = reading.read_antpos_etcd()
df_cnf = df_cnf_read[['snap2_hostname', 'snap2_chassis', 'snap2_location', 'fmc', 'pola_digitizer_channel',
                      'polb_digitizer_channel', 'arx_address', 'pola_arx_channel', 'polb_arx_channel',
                      'pola_fee', 'polb_fee']]

## ARX

In [ ]:
ma = lwa_arx.ARX()
arxcfg = []
for adr in adrs:
    try:
        dds = ma.get_all_chan_cfg(adr)
    except:
        continue
    for ch, dd in enumerate(dds):
        dd['adr-ch'] = f'{adr}-{ch}'
        arxcfg.append(dd)
    dd_astatus = {'name': f'arx{adr}', 'mp_age': 0, 'ready': dd['sig_on']}
    dd_statuses.append(dd_astatus)

# for missing ARXs?
#    dd_astatus = {'name': f'arx{adr}', 'mp_age': -1, 'ready': False}
#    dd_statuses.append(dd_astatus)

df_arx = pd.DataFrame.from_dict(arxcfg)
if any(df_arx):
    df_arx.set_index('adr-ch', inplace=True)


In [ ]:
#df_arx

## Build table functions

In [ ]:
# f-engine table build function
def build_f():
    dd_fs = []
    t_now = time.Time.now().unix
    stats = con.status_fengine()

    for snap2name in snap2names:
        snap2num = int(snap2name.lstrip('snap'))
#    lwa_feng = snap2_fengine.Snap2Fengine(snap2name)
#    lwa_fe = snap2_feng_etcd_client.Snap2FengineEtcdClient(snap2name, snap2num)

#    st, fl = lwa_feng.fpga.get_status()
        t_now = time.Time.now().unix
        if stats[snap2name] is None:
            continue
        else:
            dd_fstatus = {'name': snap2name, 'mp_age': t_now-time.Time(stats[snap2name]['fpga']['timestamp']).unix,
                          'ready': stats[snap2name]['fpga']['programmed']}
#        dd_fstatus = None
# is there a key for lwa_feng.is_connected()?

        dd_statuses.append(dd_fstatus)

        plot_fstats = None

        dd_f = {'name': snap2name}
        st = stats[snap2name]['fpga']
#    dd_f['is_connected'] = lwa_feng.is_connected()  # can SNAP be reached with ping on network?
#    dd_f['is_polling'] = lwa_fe.is_polling()   # is monitor service checking on SNAP?
        dd_f['programmed'] = st['programmed']
        dd_f['firmware'] = st['flash_firmware']
        
        dd_f['mp_age'] = t_now-time.Time(st['timestamp']).unix

        st = stats[snap2name]['sync']
        dd_f['uptime_fpga_clks'] = st['uptime_fpga_clks']

        st = stats[snap2name]['autocorr']
        dd_f['autocorr_acc_len'] = st['acc_len']

        st = stats[snap2name]['corr']
        dd_f['corr_acc_len'] = st['acc_len']

        st = stats[snap2name]['eqtvg']
        dd_f['tvg_enabled'] = st['tvg_enabled']

        st = stats[snap2name]['eq']
        dd_f['clip_count'] = st['clip_count']

        dd_fs.append(dd_f)

    df_f = pd.DataFrame.from_dict(dd_fs)
    if any(df_f):
        df_f.set_index('name', inplace=True)
        
    df_f_pane = pn.widgets.Tabulator(df_f, layout='fit_data_table', pagination='remote', page_size=20)
    return df_f_pane

# TODO: f-engine stats plot build function
def build_fstats(snap2name):
    st = stats[snap2name]['input']
    pows = np.array([v for (k,v) in st.items() if 'pow' in k])
    means = np.array([v for (k,v) in st.items() if 'mean' in k])
    rms = np.array([v for (k,v) in st.items() if 'rms' in k])
#    snap_input = [f'{snap2name}-{k.strip("rms")}' for (k,v) in st.items() if 'rms' in k]
    df_fstats = pd.DataFrame(data={'pows': pows, 'mean': means, 'snap': len(pows)*[f'{snap2name}']})
    plot_fstats = df_fstats.hvplot.scatter('pows', 'mean', by='snap', width=500, title='F-engine stats')

    return plot_fstats
    
# x-engine table build function
def build_x():
    npipeline = con.conf['xengines']['nxpipeline']
    dd_xs = []
    t_now = time.Time.now().unix
    for host in xhosts:
        for pipeline in range(npipeline):
            lwa_x = Lwa352PipelineControl(pipeline_id=pipeline, etcdhost=common.ETCD_HOST, host=host)
            if lwa_x.pipeline_is_up():
                st = lwa_x.corr.get_bifrost_status()
                t_now = time.Time.now().unix
                stats = st['stats']
                name = f'{host}-{pipeline}'
                dd_xstatus = {'name': name}
                dd_x = {'name': name}
                dd_x['gbps'] = st['gbps']
                dd_x['mp_age'] = t_now-st['time']

                if len(stats) > 1:
                    dd_xstatus['ready'] = stats['state'] == 'running'
                    if stats['state'] == 'running':
                        dd_x['curr_sample'] = stats['curr_sample']
                        dd_x['state'] = stats['state']
                dd_xs.append(dd_x)

                dd_xstatus['mp_age'] = t_now-st['time']
                dd_statuses.append(dd_xstatus)
            else:
                print(f'Pipeline for {host}:{pipeline} is not up')

    df_x = pd.DataFrame.from_dict(dd_xs)
    if any(df_x):
        df_x.set_index('name', inplace=True)
    df_x_pane = pn.widgets.Tabulator(df_x, layout='fit_data_table', pagination='remote', page_size=20)
    return df_x_pane

# Data recorder table build function
def build_dr():
    dd_ds = []
    t_now = time.Time.now().unix
    data_baselines = None
    data_spectra = None
    for drid in recorders:
        lwa_dr = mcs.Client(drid)
        summary = lwa_dr.read_monitor_point('summary')
        if summary is None:
            continue
        t_mp = summary.timestamp
        dd_dstatus = {'name': drid, 'mp_age': t_now-t_mp,
                      'ready': summary.value == 'normal'}
        dd_statuses.append(dd_dstatus)

        rx_rate = None
        rx_missing = None
        pipeline_lag = None
        if 'bifrost/rx_rate' in lwa_dr.list_monitor_points():
            rx_rate = lwa_dr.read_monitor_point('bifrost/rx_rate').value
            rx_missing = lwa_dr.read_monitor_point('bifrost/rx_missing').value
            pipeline_lag = lwa_dr.read_monitor_point('bifrost/pipeline_lag').value

        info = lwa_dr.read_monitor_point('info')

        dd_d = {'name': f'{drid}', 'rx_rate': rx_rate, 'rx_missing': rx_missing,
                'pipeline_lag': pipeline_lag, 'info': info.value}
        dd_ds.append(dd_d)
    else:
        lwa_dr = None
        
    df_d = pd.DataFrame.from_dict(dd_ds)
    if any(df_d):
        df_d.set_index('name', inplace=True)
    df_d_pane = pn.widgets.Tabulator(df_d, layout='fit_data_table', pagination='remote', page_size=20)
    return df_d_pane

In [ ]:
#df_f_pane
#df_x_pane
#df_d_pane

In [ ]:
df_status = pd.DataFrame.from_dict(dd_statuses)
df_status.set_index('name', inplace=True)

## Build dashboard

In [ ]:
# coloring and filtering functions

frequencies = np.linspace(0, 196/2, 4096//8, endpoint=False)  # get_new_corr does 8-channel average

def filter_df(antpol):
    """ Given ant-pol string input, plot the snap2 f-engine autocorr
    """

#    return df_spec[colname].hvplot.line(title=f'autocorrelation spectrum', xlabel='channels', ylabel='amplitude')
    if not antpol:
        specs = {f'no ant': np.zeros(512), 'frequencies': frequencies}
        df_spec = pd.DataFrame.from_dict(specs)
        return df_spec.hvplot.line(x='frequencies', title=f'autocorrelation spectrum',
                                   xlabel='frequencies', ylabel='amplitude')
    numpol = antpol.upper().lstrip('LWA-')  # prep input
    num = numpol[:-1]
    pol = numpol[-1:]
    snap2num, inp = mapping.antpol_to_digitizer(f'LWA-{num}', pol)
    lwa_feng = snap2_fengine.Snap2Fengine(f'snap{snap2num:02}')
    
    spec = lwa_feng.corr.get_new_corr(inp, inp).real  # 8-channel average, normalized by accumulated time/chans
    del lwa_feng
    specs = {f'antpol {antpol}': spec, 'frequencies': frequencies}
    df_spec = pd.DataFrame.from_dict(specs)
    return df_spec.hvplot.line(x='frequencies', y=f'antpol {antpol}', title=f'autocorrelation spectrum',
                               xlabel='frequencies', ylabel='amplitude')

def color_bad(val):
    """
    Colors text red if bad.
    Made for status df, so False and mp_age > 100 are "bad"
    """
    color = 'red' if (val is False or val > 100) else 'black'
    return 'color: %s' % color

#col_filter = pn.widgets.TextInput(placeholder='Enter a f-eng input for snap01')
col_filter = pn.widgets.TextInput(placeholder='Plot f-eng for ant-pol name (e.g., 240a)')

In [ ]:
# using Tabulator
df_status_pane = pn.widgets.Tabulator(df_status, layout='fit_data_table', pagination='remote', page_size=20)
#.style.applymap(color_bad)
df_cnf_pane = pn.widgets.Tabulator(df_cnf, layout='fit_data_table', pagination='remote', page_size=20)
df_arx_pane = pn.widgets.Tabulator(df_arx, layout='fit_data_table', pagination='remote', page_size=20)

#df_status_pane.add_filter(slider, 'mp_age')
#df_status_pane_sel = pn.Column(slider, df_status_pane)

In [ ]:
pf = pn.param.ParamFunction(build_f, lazy=True)
px = pn.param.ParamFunction(build_x, lazy=True)
pdr = pn.param.ParamFunction(build_dr, lazy=True)

In [ ]:
dashboard_title = pn.panel('## OVRO-LWA dashboard')
tabs = pn.Tabs(('Status', df_status_pane),
               ('Cabling', df_cnf_pane),
               ('ARX', df_arx_pane),
               ('F-engine', pf),
               ('X-engine', px),
               ('Data recorder', pdr), dynamic=True, active=0)  # dynamic=True to speed things up?
header_f = pn.pane.Markdown('##F-engine plots', width=1000)
plot_spec_pane = pn.Column(pn.Column(col_filter), pn.panel(pn.bind(filter_df, col_filter)), width=1000)
mini_dashboard = pn.Column(dashboard_title, tabs, header_f,
                           plot_spec_pane, width=1000)

In [ ]:
mini_dashboard.servable(title='OVRO-LWA dashboard')